# PLC siemens + realsense D455 rect depth matching

## Snap7 PLC connection

In [ ]:
#common variables
found_part_num=0

## PLC class import

In [8]:
import sys
sys.path.insert(1, '../python')
import plc

## PLC class debug

In [3]:
import snap7
import sys
import logging
from logging import handlers
from logging.handlers import RotatingFileHandler
import traceback
import time
import threading
from snap7.util import *
import yaml
import os
csd = '../python' #os.path.dirname(os.path.abspath(__file__))
config = yaml.safe_load(open(csd+"/config.yml"))

# TODO found_part_num
#found_part_num = 0
camera_db_num = config['plc']['camera_db_num']
reconnect_timeout = config['plc']['reconnect_timeout']

class PLC(threading.Thread):
    def __init__(self, plc_ip):
        #init
        threading.Thread.__init__(self, args=(), name=plc_ip, kwargs=None)
        self.plc_ip = plc_ip
        self.found_part_num = 0
        self.snapshotReq = False
        self.plc_ip = plc_ip
        self.logger = logging.getLogger("_plc_.client")
        #logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))
        logging.basicConfig(level=config['logger']['level'],
                            handlers=[logging.StreamHandler(sys.stdout),
                                      logging.handlers.RotatingFileHandler(config['logger']['debug_file'],
                                                                           maxBytes=(1048576 * 5),
                                                                           backupCount=7)],
                            format=config['logger']['format'])
        self.snap7client = snap7.client.Client()
        self.connection_ok = False
        self.unreachable_time = 0
        self.connect_postpone = False

    def get_bool(self, db_number, offsetbyte, offsetbit):
        return snap7.util.get_bool(self.db_read(db_number, offsetbyte, 1), 0, offsetbit)

    def set_usint(self, db_number, offsetbyte, tag_value):
        tag_data = bytearray(1)
        snap7.util.set_usint(tag_data, 0, tag_value)
        self.snap7client.db_write(db_number, offsetbyte, tag_data)
        return True

    def db_read(self, db_number, offsetbyte, len_arr):
        return self.snap7client.db_read(db_number, offsetbyte, len_arr)

    def db_write(self, db_number, offsetbyte, tag_data):
        return self.snap7client.db_write(db_number, offsetbyte, tag_data)

    def run(self):
        self.logger.info(f"Connection with PLC {self.plc_ip} started")
        cur_thread = threading.current_thread()
        # Основной цикл
        while getattr(cur_thread, "do_run", True):
            try:
                time.sleep(1.2)
                if self.unreachable_time == 0 or (time.time() - self.unreachable_time) > reconnect_timeout:
                    if not self.snap7client.get_connected():
                        # Подключение к контроллеру ...
                        try:
                            self.connection_ok = False
                            self.logger.info(f"Подключение к контроллеру {self.plc_ip}...")
                            self.snap7client.connect(self.plc_ip, 0, 1)
                        except Exception as error:
                            self.logger.error(f"Не удалось подключиться к контроллеру: {self.plc_ip}\n"
                                              f"Ошибка {str(error)} {traceback.format_exc()}")
                            snap7.client.logger.disabled = True
                            self.unreachable_time = time.time()
                    else:
                        if not self.connection_ok:
                            self.connection_ok = True
                            self.unreachable_time = 0
                            self.logger.info(f"Соединение открыто {self.plc_ip}")
                            snap7.client.logger.disabled = False
                        try:
                            if not self.snapshotReq:
                                self.snapshotReq = self.get_bool(db_number=camera_db_num, offsetbyte=0, offsetbit=0)
                                if self.snapshotReq:
                                    self.logger.info(f"Строб съёмки пришёл {self.snapshotReq}")
                        except Exception as error:
                            self.logger.error(f"Не удалось считать строб съёмки: DB{camera_db_num}.DBX0.0\n"
                                              f"Ошибка {str(error)} {traceback.format_exc()}")
                            self.snap7client.disconnect()


                        if self.snapshotReq:
                            if self.found_part_num > 0:
                               self.logger.info(f"Запись результата распознования - номер найденной детали - {self.found_part_num}")
                               try:
                                   self.set_usint(db_number=camera_db_num, offsetbyte=1, tag_value=self.found_part_num)
                                   self.found_part_num = 0
                               except Exception as error:
                                   self.logger.error(f"Не удалось записать результат съёмки: DB{camera_db_num}.DBB1\n"
                                                     f"Ошибка {str(error)} {traceback.format_exc()}")
                                   self.snap7client.disconnect()
            except Exception as error:
                self.logger.error(f"Не удалось обработать цикл класса plc\n"
                                  f"Ошибка {str(error)} {traceback.format_exc()}")

In [3]:
my_plc = PLC('192.168.1.101')

In [4]:
my_plc.start()

## Part detection with Intel realsense D455 and OpenCV

In [1]:
#var huck
found_part_num = 0
teach_part_num = 0

#separate file - configurations of parts and settings
tolerance = 0.05
resolution_x,resolution_y=640,480
#resolution_x,resolution_y=1280,720
fps=30
parts=[ dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ А',h=212,w=89,Zavg=400),
        dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ B',h=208,w=87,Zavg=400),
        dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ C',h=208,w=80.6,Zavg=400),
        dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ D',h=208,w=80,Zavg=400),
        dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ E',h=528,w=268,Zavg=400),
        dict(code='17115.2900.77',part_name='Переходник',step_name='Установ А',h=273,w=206,Zavg=400),
        dict(code='17115.2900.77',part_name='Переходник',step_name='Установ B',h=209.03,w=206,Zavg=400)]

## License: Apache 2.0. See LICENSE file in root directory.
## Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

###############################################
##      Open CV and Numpy integration        ##
###############################################

import pyrealsense2 as rs
import numpy as np
#import matplotlib.pyplot as plt 
import cv2
#import opencv_jupyter_ui as jcv2


import cv2
#from transform import hough_windowed_rectangle, hough_cricles
#from utils import plot_hough_space, graph_output, graph_rectangles_and_circles, countMoney
#from skimage.io import imshow
#import matplotlib.pyplot as plt


template = cv2.imread("part1.png")



# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
#depth_sensor = device.first_depth_sensor()
device_product_line = str(device.get_info(rs.camera_info.product_line))
print('device_product_line',device_product_line)

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)
align = rs.align(rs.stream.color)
config.enable_stream(rs.stream.depth, resolution_x, resolution_y, rs.format.z16, fps)
config.enable_stream(rs.stream.color, resolution_x, resolution_y, rs.format.bgr8, fps)

# Start streaming
pipeline.start(config)
try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        
        color_image = np.asanyarray(color_frame.get_data())
        
        frames = []
        for x in range(15):
            frameset = pipeline.wait_for_frames()
            frames.append(frameset.get_depth_frame())
        
        hole_filling = rs.hole_filling_filter()
        for x in range(15):
            frame = hole_filling.process(frames[x])
            frames[x] = np.asanyarray(frame.get_data())
        
        depth_image = np.min(frames, axis=0)
        # print(np.min(depth_image),np.max(depth_image))
        # depth_image = depth_image.astype(dtype=np.uint8)
        # #depth_image = cv2.cvtColor(depth_image,cv2.COLOR_BGR2GRAY)
        # depth_image = cv2.adaptiveThreshold(depth_image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,51,9)
        # #depth_image = cv2.adaptiveThreshold(depth_image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
        # #depth_image = cv2.adaptiveThreshold(depth_image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)

        # Calculate the min along the time axis
        #depth_image = np.min(frames, axis=0).astype(dtype=np.uint16)
        
        ##hight normalization for template search
        #resultimage = np.zeros((resolution_x,resolution_y))
        #depth_image = cv2.normalize(depth_image, resultimage, 0, 100, cv2.NORM_MINMAX)
        
        depth_image = depth_image.astype(dtype=np.uint16)
        #depth_image = cv2.medianBlur(depth_image,5)
       
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_image = cv2.convertScaleAbs(depth_image, alpha=0.3)
        depth_image2 = depth_image.copy()
        
        part_area = 10
        x, y = resolution_x//2-53, resolution_y//2+27
        w, h = resolution_x//part_area, resolution_y//part_area  
        crop_img = depth_image[x-w:x+w, y-h:y+h]
        #crop_img[crop_img<10] = 255
        min_hight_color = np.min(crop_img)
        depth_image[depth_image<min_hight_color]=255
        #depth_image2[depth_image2<min_hight_color]=255
        print('max_hight_color', min_hight_color)
        depth_image = depth_image - min_hight_color
        
        
        cv2.rectangle(depth_image, (x-w,y-h), (x+w,y+h), (255,0,0), 1, 16)

        
        #constants
        centre_x,centre_y = resolution_x//2-53, resolution_y//2+27
        trust_interval=[320,490]
        min_spot=20
        
        x1,y1 = int(centre_x-min_spot//2), int(centre_y-min_spot//2)
        x2,y2 = int(centre_x+min_spot//2), int(centre_y+min_spot//2)
        cropped_image = depth_image[x1:x2,y1:y2]
        Cmean = np.mean(cropped_image)       
        
        #monochrome view
        #depth_image[depth_image<Cmean-15]=0
        #depth_image[depth_image>Cmean+15]=0        
        #depth_image[depth_image>0]=255        
        
                
        #contours = cv2.findContours(depth_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        contours, _  = cv2.findContours(depth_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #print(len(contours))
        bboxes = []
        rboxes = []
        cnts = []
        k=-1.5
        b=565
        Zavg = k*Cmean+b
        for cnt in contours:
            ## Skip non-central area
            if cv2.pointPolygonTest(cnt, (centre_x,centre_y), True)<0:
                continue
            ## Get the stright bounding rect
            bbox = cv2.boundingRect(cnt)
            x,y,w,h = bbox
            if w<30 or h < 30 or w*h < 2000 or w > 500:
                continue

            ## Draw rect
            cv2.rectangle(depth_image, (x,y), (x+w,y+h), (255,0,0), 1, 16)

            ## Get the rotated rect
            rbox = cv2.minAreaRect(cnt)
            (x1,y1), (cw,ch), rot_angle = rbox
            x1=int(x1)
            y1=int(y1)
            cw=int(cw)
            ch=int(ch)
            #print ((x1,y1), (cw,ch))
            ## Draw rotated rect
            #!!!!!!!!!not worked!!!!!
            #cv2.rectangle(depth_image, (x1,y1), (x1+cw,y1+ch), (255,0,0), 1, 16)
            #print("rot_angle:", rot_angle)  

            ## backup 
            bboxes.append(bbox)
            rboxes.append(rbox)
            cnts.append(cnt)
            

            # cv2.putText(depth_colormap, f"Zavg {round(Zavg, 1)} mm", (int(x1 - 20), int(y1 - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)
            # cv2.putText(color_image, f"Zavg {round(Zavg, 1)} mm", (int(x1 - 20), int(y1 - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)
            #print(f'w {hw[1]} h {hw[0]} Zavg {Zavg}')
            pk=0.003125
            pb=-0.125
            part_w = cw*(pk*Zavg+pb)
            part_h = ch*(pk*Zavg+pb)
            #print(f'part_w {part_w} part_h {part_h}')
            cv2.putText(depth_image, f"{round(part_h, 0)} x {round(part_w, 0)} mm", (int(x1 - 20), int(y1 - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)
            cv2.putText(color_image, f"{round(part_h, 0)} x {round(part_w, 0)} mm", (int(x1 - 20), int(y1 - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)
            
        depth_colormap = cv2.applyColorMap(depth_image, cv2.COLORMAP_JET)
        h, w = template.shape[:2]
        methods = [cv2.TM_SQDIFF_NORMED, cv2.TM_CCORR_NORMED, cv2.TM_CCOEFF_NORMED]
        res = cv2.matchTemplate(depth_colormap, template, methods[2])
        threshold = 0.8
        loc = np.where(res >= threshold)  # Coordinates y, x where the matching degree is greater than %80
        print(loc)
        for pt in zip(*loc[::-1]):  # * Indicates optional parameters
            right_bottom = (pt[0] + w, pt[1] + h)
            cv2.rectangle(depth_colormap, pt, right_bottom, (0, 0, 255), 1)

        
        depth_colormap2 = cv2.applyColorMap(depth_image2, cv2.COLORMAP_JET)
                
        # rectangle = np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2],[x1,y1]], np.int32)
        # depth_colormap =cv2.polylines(depth_colormap, [rectangle], False, (255,0,0), thickness=1)
        # color_image =cv2.polylines(color_image, [rectangle], False, (255,0,0), thickness=1)
        
        


        last_found_part_num = found_part_num 
        found_part_num = 0
        for num, part in enumerate(parts):
            h, w, Zavg = part['h'], part['w'], part['Zavg']
            part_name, step_name = part['part_name'],part['step_name']
            if trust_interval[0]<Zavg<trust_interval[1]:
                x1,y1 = int(centre_x-w//2), int(centre_y-h//2)
                x2,y2 = int(centre_x+w//2), int(centre_y+h//2)
                cropped_image = depth_colormap[x1:x2,y1:y2]
                Cavg = np.mean(cropped_image)
                CurZavg = k*Cavg+b
                
                if Zavg*(1-tolerance)<CurZavg<Zavg*(1+tolerance):
                    if found_part_num==0:
                        found_part_name=part_name
                        found_part_step=step_name
                        found_part_num=num
                        rectangle = np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2],[x1,y1]], np.int32)
                       
                        depth_colormap =cv2.polylines(depth_colormap, [rectangle], False, (0,255,0), thickness=1)
                        color_image =cv2.polylines(color_image, [rectangle], False, (0,255,0), thickness=1)
                        if last_found_part_num != found_part_num:
                            print(f'Part detected: {part_name} {step_name}')
                    else:
                        print(f'warning! several parts detected {part_name} {step_name} and {found_part_name} {found_part_step}')
                        found_part_num=0
                        break
            else:
                print(f'warning! part {num} have wrong Zavg {Zavg}. Trust interval is {trust_interval}')

        #diagonal draw 
        #depth_colormap =cv2.polylines(depth_colormap, [np.array([[0,0],[resolution_x,resolution_y]], np.int32)], False, (0,255,0), thickness=1)
        #color_image =cv2.polylines(color_image, [np.array([[0,0],[resolution_x,resolution_y]], np.int32)], False, (0,255,0), thickness=1)
         
        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape
        
        

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((depth_colormap,resized_color_image))
        else:
            #images = np.hstack((depth_colormap, color_image))
            images = np.hstack((depth_colormap, depth_colormap2))

        # Show images
        cv2.startWindowThread()
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
        
finally: 

    # Stop streaming
    pipeline.stop() 

ImportError: libusb-1.0.so.0: cannot open shared object file: No such file or directory

In [ ]:
#cv2.destroyAllWindows()

In [ ]:
def template_demo():
    tpl = cv.imread("./files/images/match2.png")
    target = cv.imread("./files/images/match1.png")

    methods = [cv.TM_SQDIFF_NORMED, cv.TM_CCORR_NORMED, cv.TM_CCOEFF_NORMED]  # Only test these three template matching methods
    th, tw = tpl.shape[:2] #Take length and width

    for md in methods:
        print(md)
        result = cv.matchTemplate(target, tpl, md)  # Get matching results
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
        if md == cv.TM_SQDIFF_NORMED:  # cv.TM_SQDIFF_NORMED is the most similar when it is the smallest, most similar when the others are the largest
            tl = min_loc
        else:
            tl = max_loc

        br = (tl[0] + tw, tl[1] + th)
        cv.rectangle(target, tl, br, (0, 0, 255), 2)  # tl is the coordinates of the upper left corner, br is the coordinates of the lower right corner, so as to draw a rectangle
        cv.imshow("match-"+np.str(md), target)

In [9]:
def template_demo_more():
    img_rgb = cv.imread('./files/images/temp1.jpg')
    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    template = cv.imread('./files/images/temp2.jpg', 0)
    h, w = template.shape[:2]

    res = cv.matchTemplate(img_gray, template, cv.TM_CCOEFF_NORMED)
    threshold = 0.8

    loc = np.where(res >= threshold)  # Coordinates y, x where the matching degree is greater than %80
    print(loc)
    for pt in zip(*loc[::-1]):  # * Indicates optional parameters
        right_bottom = (pt[0] + w, pt[1] + h)
        cv.rectangle(img_rgb, pt, right_bottom, (0, 0, 255), 2)
        cv.imshow("more",img_rgb)
        